In [ ]:
"""
Solana Memecoin Sniping Bot
Integrates Dexscreener API and Twitter trends for automated memecoin sniping
"""

import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import pytz
import logging
import sys
import os
from typing import Optional, Dict, Any, List, Tuple
import warnings
import json
import asyncio
from dataclasses import dataclass

# Suppress pandas warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

# HTTP requests
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Solana blockchain libraries
try:
    from solders.keypair import Keypair
    from solders.pubkey import Pubkey
    from solders.rpc.responses import GetTransactionResp
    from solders.transaction import Transaction
    from solana.rpc.async_api import AsyncClient
    from solana.rpc.commitment import Confirmed, Finalized
    from solana.rpc.types import TxOpts
    import base58
    SOLANA_AVAILABLE = True
except ImportError:
    SOLANA_AVAILABLE = False
    logging.warning("Solana libraries not available. Install: pip install solana solders")

# Jupiter DEX aggregator for swaps
try:
    import jupiter_python_sdk
    JUPITER_AVAILABLE = True
except ImportError:
    JUPITER_AVAILABLE = False
    logging.warning("Jupiter SDK not available. Install: pip install jupiter-python-sdk")

# Twitter/X API
TWITTER_LIB = None
try:
    from twitter_api_v2 import TwitterApi
    TWITTER_AVAILABLE = True
    TWITTER_LIB = 'twitter_api_v2'
except ImportError:
    try:
        import tweepy
        TWITTER_AVAILABLE = True
        TWITTER_LIB = 'tweepy'
    except ImportError:
        TWITTER_AVAILABLE = False
        logging.warning("Twitter API libraries not available. Install: pip install tweepy or twitter-api-v2")

# Observability
from prometheus_client import Counter, Gauge, Histogram, start_http_server
import json

# Database (TimescaleDB) - reuse from existing code
try:
    import psycopg2
    from psycopg2.extras import execute_values
    from psycopg2.pool import SimpleConnectionPool
    DB_AVAILABLE = True
except ImportError:
    DB_AVAILABLE = False

# Configure logging
def setup_logging(level=logging.INFO):
    """Setup logging"""
    logger = logging.getLogger()
    logger.setLevel(level)
    logger.handlers.clear()
    
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler = logging.StreamHandler(sys.stdout)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger

logger = setup_logging()

# Prometheus metrics
snipe_attempts = Counter('snipe_attempts_total', 'Total snipe attempts', ['status'])
snipe_success = Counter('snipe_success_total', 'Successful snipes', ['token'])
snipe_failures = Counter('snipe_failures_total', 'Failed snipes', ['reason'])
active_positions = Gauge('active_positions', 'Number of active positions')
token_score = Gauge('token_score', 'Token score for sniping', ['token_address'])
twitter_mentions = Counter('twitter_mentions_total', 'Twitter mentions tracked', ['token'])


@dataclass
class TokenInfo:
    """Token information from Dexscreener"""
    address: str
    name: str
    symbol: str
    price_usd: float
    price_change_24h: float
    volume_24h: float
    liquidity: float
    fdv: float  # Fully diluted valuation
    market_cap: float
    pair_created_at: datetime
    dex_id: str
    chain_id: str
    pair_address: str
    score: float = 0.0  # Calculated score for sniping


@dataclass
class TwitterTrend:
    """Twitter trend information"""
    token_address: str
    mentions_count: int
    engagement_rate: float
    sentiment_score: float  # -1 to 1
    trending_keywords: List[str]
    last_updated: datetime


class DexscreenerAPI:
    """Dexscreener API client for token discovery"""
    
    def __init__(self, base_url: str = "https://api.dexscreener.com/latest/dex"):
        self.base_url = base_url
        self.session = self._create_session()
        self.rate_limit_delay = 0.5  # Delay between requests
        
    def _create_session(self):
        """Create requests session with retry strategy"""
        session = requests.Session()
        retry_strategy = Retry(
            total=3,
            backoff_factor=1,
            status_forcelist=[429, 500, 502, 503, 504],
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount("http://", adapter)
        session.mount("https://", adapter)
        return session
    
    def get_tokens_by_chain(self, chain: str = "solana", limit: int = 100) -> List[Dict]:
        """Get trending tokens on a specific chain"""
        try:
            time.sleep(self.rate_limit_delay)
            url = f"{self.base_url}/tokens/{chain}"
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            if 'pairs' in data:
                return data['pairs'][:limit]
            return []
        except Exception as e:
            logger.error(f"Error fetching tokens from Dexscreener: {e}")
            return []
    
    def get_token_info(self, token_address: str, chain: str = "solana") -> Optional[Dict]:
        """Get detailed information about a specific token"""
        try:
            time.sleep(self.rate_limit_delay)
            url = f"{self.base_url}/tokens/{token_address}"
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            if 'pairs' in data and len(data['pairs']) > 0:
                return data['pairs'][0]
            return None
        except Exception as e:
            logger.error(f"Error fetching token info for {token_address}: {e}")
            return None
    
    def search_tokens(self, query: str, chain: str = "solana") -> List[Dict]:
        """Search for tokens by name or symbol"""
        try:
            time.sleep(self.rate_limit_delay)
            url = f"{self.base_url}/search"
            params = {"q": query}
            response = self.session.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            if 'pairs' in data:
                # Filter by chain
                return [pair for pair in data['pairs'] if pair.get('chainId') == chain]
            return []
        except Exception as e:
            logger.error(f"Error searching tokens: {e}")
            return []
    
    def parse_token_info(self, pair_data: Dict) -> Optional[TokenInfo]:
        """Parse Dexscreener pair data into TokenInfo"""
        try:
            return TokenInfo(
                address=pair_data.get('baseToken', {}).get('address', ''),
                name=pair_data.get('baseToken', {}).get('name', ''),
                symbol=pair_data.get('baseToken', {}).get('symbol', ''),
                price_usd=float(pair_data.get('priceUsd', 0)),
                price_change_24h=float(pair_data.get('priceChange', {}).get('h24', 0)),
                volume_24h=float(pair_data.get('volume', {}).get('h24', 0)),
                liquidity=float(pair_data.get('liquidity', {}).get('usd', 0)),
                fdv=float(pair_data.get('fdv', 0)),
                market_cap=float(pair_data.get('marketCap', 0)),
                pair_created_at=datetime.fromtimestamp(pair_data.get('pairCreatedAt', 0) / 1000),
                dex_id=pair_data.get('dexId', ''),
                chain_id=pair_data.get('chainId', ''),
                pair_address=pair_data.get('pairAddress', '')
            )
        except Exception as e:
            logger.error(f"Error parsing token info: {e}")
            return None


class TwitterTrendAnalyzer:
    """Twitter/X trend analyzer for memecoin detection"""
    
    def __init__(self, api_key: str = None, api_secret: str = None, 
                 bearer_token: str = None, access_token: str = None, 
                 access_token_secret: str = None):
        self.api_key = api_key
        self.api_secret = api_secret
        self.bearer_token = bearer_token
        self.access_token = access_token
        self.access_token_secret = access_token_secret
        self.client = None
        self._initialize_client()
        
    def _initialize_client(self):
        """Initialize Twitter API client"""
        if not TWITTER_AVAILABLE or not TWITTER_LIB:
            logger.warning("Twitter libraries not available")
            return
            
        try:
            if TWITTER_LIB == 'tweepy':
                import tweepy
                if self.bearer_token:
                    self.client = tweepy.Client(bearer_token=self.bearer_token)
                elif self.api_key and self.api_secret:
                    auth = tweepy.OAuthHandler(self.api_key, self.api_secret)
                    if self.access_token and self.access_token_secret:
                        auth.set_access_token(self.access_token, self.access_token_secret)
                    self.client = tweepy.API(auth)
            else:
                # Using twitter-api-v2
                from twitter_api_v2 import TwitterApi
                self.client = TwitterApi(bearer_token=self.bearer_token)
        except Exception as e:
            logger.error(f"Failed to initialize Twitter client: {e}")
            self.client = None
    
    def search_tweets(self, query: str, max_results: int = 100) -> List[Dict]:
        """Search for tweets matching query"""
        if not self.client:
            return []
            
        try:
            if TWITTER_LIB == 'tweepy':
                tweets = self.client.search_recent_tweets(
                    query=query,
                    max_results=min(max_results, 100),
                    tweet_fields=['created_at', 'public_metrics', 'author_id']
                )
                if tweets.data:
                    return [tweet.data for tweet in tweets.data]
            else:
                # Using twitter-api-v2
                response = self.client.search_recent_tweets(
                    query=query,
                    max_results=max_results
                )
                if response.data:
                    return response.data
            return []
        except Exception as e:
            logger.error(f"Error searching tweets: {e}")
            return []
    
    def analyze_token_trend(self, token_symbol: str, token_address: str) -> TwitterTrend:
        """Analyze Twitter trends for a specific token"""
        if not self.client:
            return TwitterTrend(
                token_address=token_address,
                mentions_count=0,
                engagement_rate=0.0,
                sentiment_score=0.0,
                trending_keywords=[],
                last_updated=datetime.now()
            )
        
        # Search for token mentions
        queries = [
            f"${token_symbol}",
            f"#{token_symbol}",
            token_symbol,
            token_address[:8]  # Partial address
        ]
        
        total_mentions = 0
        total_engagement = 0
        keywords = []
        
        for query in queries:
            try:
                tweets = self.search_tweets(query, max_results=50)
                total_mentions += len(tweets)
                
                for tweet in tweets:
                    # Calculate engagement (likes + retweets + replies)
                    if 'public_metrics' in tweet:
                        metrics = tweet['public_metrics']
                        engagement = metrics.get('like_count', 0) + \
                                    metrics.get('retweet_count', 0) + \
                                    metrics.get('reply_count', 0)
                        total_engagement += engagement
                    
                    # Extract keywords
                    if 'text' in tweet:
                        text = tweet['text'].lower()
                        keywords.extend([w for w in text.split() if len(w) > 3])
            except Exception as e:
                logger.debug(f"Error analyzing query {query}: {e}")
        
        # Calculate sentiment (simplified - can be enhanced with NLP)
        sentiment_score = 0.0  # Placeholder - implement sentiment analysis
        
        # Calculate engagement rate
        engagement_rate = total_engagement / max(total_mentions, 1)
        
        # Get trending keywords
        from collections import Counter
        keyword_counts = Counter(keywords)
        trending_keywords = [word for word, count in keyword_counts.most_common(10)]
        
        twitter_mentions.labels(token=token_symbol).inc(total_mentions)
        
        return TwitterTrend(
            token_address=token_address,
            mentions_count=total_mentions,
            engagement_rate=engagement_rate,
            sentiment_score=sentiment_score,
            trending_keywords=trending_keywords,
            last_updated=datetime.now()
        )


class SolanaTrader:
    """Solana blockchain trader using Jupiter DEX aggregator"""
    
    def __init__(self, private_key: str, rpc_url: str = "https://api.mainnet-beta.solana.com"):
        if not SOLANA_AVAILABLE:
            raise ImportError("Solana libraries not available")
        
        self.private_key = private_key
        self.rpc_url = rpc_url
        self.keypair = None
        self.client = None
        self._initialize()
        
    def _initialize(self):
        """Initialize Solana client and keypair"""
        try:
            # Load keypair from private key
            if isinstance(self.private_key, str):
                # Assume base58 encoded private key
                private_key_bytes = base58.b58decode(self.private_key)
                self.keypair = Keypair.from_bytes(private_key_bytes)
            else:
                self.keypair = Keypair.from_bytes(self.private_key)
            
            # Initialize async client
            self.client = AsyncClient(self.rpc_url)
            logger.info(f"Solana trader initialized. Public key: {self.keypair.pubkey()}")
        except Exception as e:
            logger.error(f"Failed to initialize Solana trader: {e}")
            raise
    
    async def get_balance(self) -> float:
        """Get SOL balance"""
        try:
            response = await self.client.get_balance(self.keypair.pubkey())
            return response.value / 1e9  # Convert lamports to SOL
        except Exception as e:
            logger.error(f"Error getting balance: {e}")
            return 0.0
    
    async def get_token_balance(self, token_address: str) -> float:
        """Get SPL token balance"""
        try:
            token_pubkey = Pubkey.from_string(token_address)
            # This requires additional implementation for SPL token accounts
            # Simplified for now
            return 0.0
        except Exception as e:
            logger.error(f"Error getting token balance: {e}")
            return 0.0
    
    async def swap_tokens(self, input_mint: str, output_mint: str, 
                         amount: float, slippage_bps: int = 100) -> Optional[str]:
        """Swap tokens using Jupiter DEX aggregator"""
        if not JUPITER_AVAILABLE:
            logger.error("Jupiter SDK not available")
            return None
        
        try:
            # Jupiter API integration
            jupiter_url = "https://quote-api.jup.ag/v6/quote"
            
            # Get quote
            quote_params = {
                "inputMint": input_mint,
                "outputMint": output_mint,
                "amount": int(amount * 1e9),  # Convert SOL to lamports
                "slippageBps": slippage_bps
            }
            
            response = requests.get(jupiter_url, params=quote_params, timeout=10)
            response.raise_for_status()
            quote = response.json()
            
            if 'error' in quote:
                logger.error(f"Jupiter quote error: {quote['error']}")
                return None
            
            # Execute swap (simplified - full implementation requires transaction building)
            logger.info(f"Quote received: {quote.get('outAmount', 0)} output tokens")
            
            # TODO: Build and send transaction
            # This requires full Jupiter swap integration
            
            return None  # Return transaction signature when implemented
        except Exception as e:
            logger.error(f"Error swapping tokens: {e}")
            return None
    
    async def snipe_token(self, token_address: str, sol_amount: float, 
                         slippage_bps: int = 500) -> Optional[str]:
        """Snipe a token (buy immediately after listing)"""
        try:
            # SOL mint address
            sol_mint = "So11111111111111111111111111111111111111112"
            
            # Execute swap
            tx_signature = await self.swap_tokens(
                input_mint=sol_mint,
                output_mint=token_address,
                amount=sol_amount,
                slippage_bps=slippage_bps
            )
            
            if tx_signature:
                logger.info(f"Successfully sniped {token_address}. TX: {tx_signature}")
                snipe_success.labels(token=token_address[:8]).inc()
                return tx_signature
            else:
                logger.error(f"Failed to snipe {token_address}")
                snipe_failures.labels(reason='swap_failed').inc()
                return None
        except Exception as e:
            logger.error(f"Error sniping token: {e}")
            snipe_failures.labels(reason='exception').inc()
            return None


class MemecoinSniperBot:
    """Main bot for memecoin sniping based on Dexscreener and Twitter trends"""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        
        # Initialize components
        self.dexscreener = DexscreenerAPI()
        self.twitter = TwitterTrendAnalyzer(
            api_key=config.get('twitter_api_key'),
            api_secret=config.get('twitter_api_secret'),
            bearer_token=config.get('twitter_bearer_token'),
            access_token=config.get('twitter_access_token'),
            access_token_secret=config.get('twitter_access_token_secret')
        )
        
        if config.get('solana_private_key'):
            self.trader = SolanaTrader(
                private_key=config['solana_private_key'],
                rpc_url=config.get('solana_rpc_url', 'https://api.mainnet-beta.solana.com')
            )
        else:
            self.trader = None
            logger.warning("No Solana private key provided - trading disabled")
        
        # Trading parameters
        self.min_liquidity = config.get('min_liquidity', 50000)  # USD
        self.max_market_cap = config.get('max_market_cap', 10000000)  # USD
        self.min_volume_24h = config.get('min_volume_24h', 10000)  # USD
        self.max_age_hours = config.get('max_age_hours', 24)  # Token age
        self.min_twitter_mentions = config.get('min_twitter_mentions', 10)
        self.snipe_amount_sol = config.get('snipe_amount_sol', 0.1)  # SOL per snipe
        self.max_positions = config.get('max_positions', 5)
        self.slippage_bps = config.get('slippage_bps', 500)  # 5% slippage
        
        # State
        self.active_positions = {}  # token_address -> position info
        self.scanned_tokens = set()
        
        logger.info("Memecoin Sniper Bot initialized")
    
    def calculate_token_score(self, token: TokenInfo, twitter_trend: TwitterTrend) -> float:
        """Calculate score for a token based on multiple factors"""
        score = 0.0
        
        # Liquidity score (higher is better, but not too high)
        if token.liquidity >= self.min_liquidity:
            liquidity_score = min(token.liquidity / 1000000, 1.0) * 20
            score += liquidity_score
        
        # Volume score
        if token.volume_24h >= self.min_volume_24h:
            volume_score = min(token.volume_24h / 1000000, 1.0) * 25
            score += volume_score
        
        # Price change score (momentum)
        if token.price_change_24h > 0:
            price_score = min(token.price_change_24h / 100, 1.0) * 20
            score += price_score
        
        # Market cap score (prefer lower for memecoins)
        if 0 < token.market_cap <= self.max_market_cap:
            market_cap_score = (1 - token.market_cap / self.max_market_cap) * 15
            score += market_cap_score
        
        # Age score (newer is better)
        age_hours = (datetime.now() - token.pair_created_at).total_seconds() / 3600
        if age_hours <= self.max_age_hours:
            age_score = (1 - age_hours / self.max_age_hours) * 10
            score += age_score
        
        # Twitter trend score
        if twitter_trend.mentions_count >= self.min_twitter_mentions:
            mention_score = min(twitter_trend.mentions_count / 100, 1.0) * 5
            score += mention_score
        
        if twitter_trend.engagement_rate > 0:
            engagement_score = min(twitter_trend.engagement_rate / 100, 1.0) * 5
            score += engagement_score
        
        token.score = score
        token_score.labels(token_address=token.address[:8]).set(score)
        
        return score
    
    def filter_tokens(self, tokens: List[TokenInfo]) -> List[TokenInfo]:
        """Filter tokens based on criteria"""
        filtered = []
        
        for token in tokens:
            # Basic filters
            if token.liquidity < self.min_liquidity:
                continue
            if token.market_cap > self.max_market_cap:
                continue
            if token.volume_24h < self.min_volume_24h:
                continue
            
            # Age filter
            age_hours = (datetime.now() - token.pair_created_at).total_seconds() / 3600
            if age_hours > self.max_age_hours:
                continue
            
            # Already scanned
            if token.address in self.scanned_tokens:
                continue
            
            filtered.append(token)
        
        return filtered
    
    async def scan_and_score_tokens(self, limit: int = 50) -> List[Tuple[TokenInfo, TwitterTrend, float]]:
        """Scan Dexscreener for tokens and score them"""
        logger.info(f"Scanning Dexscreener for top {limit} tokens...")
        
        # Get tokens from Dexscreener
        pairs_data = self.dexscreener.get_tokens_by_chain("solana", limit=limit)
        
        # Parse token info
        tokens = []
        for pair_data in pairs_data:
            token = self.dexscreener.parse_token_info(pair_data)
            if token:
                tokens.append(token)
        
        # Filter tokens
        filtered_tokens = self.filter_tokens(tokens)
        logger.info(f"Found {len(filtered_tokens)} tokens matching criteria")
        
        # Score tokens with Twitter analysis
        scored_tokens = []
        for token in filtered_tokens:
            try:
                # Analyze Twitter trends
                twitter_trend = self.twitter.analyze_token_trend(token.symbol, token.address)
                
                # Calculate score
                score = self.calculate_token_score(token, twitter_trend)
                
                scored_tokens.append((token, twitter_trend, score))
                self.scanned_tokens.add(token.address)
                
                logger.debug(f"Token {token.symbol} ({token.address[:8]}): Score={score:.2f}, "
                           f"Liquidity=${token.liquidity:,.0f}, Mentions={twitter_trend.mentions_count}")
            except Exception as e:
                logger.error(f"Error processing token {token.address}: {e}")
        
        # Sort by score
        scored_tokens.sort(key=lambda x: x[2], reverse=True)
        
        return scored_tokens
    
    async def execute_snipe(self, token: TokenInfo) -> bool:
        """Execute a snipe on a token"""
        if not self.trader:
            logger.error("Trader not initialized - cannot execute snipe")
            return False
        
        if len(self.active_positions) >= self.max_positions:
            logger.warning(f"Max positions reached ({self.max_positions})")
            return False
        
        # Check balance
        balance = await self.trader.get_balance()
        if balance < self.snipe_amount_sol:
            logger.warning(f"Insufficient balance: {balance} SOL < {self.snipe_amount_sol} SOL")
            return False
        
        logger.info(f"Executing snipe on {token.symbol} ({token.address})")
        snipe_attempts.labels(status='attempting').inc()
        
        # Execute snipe
        tx_signature = await self.trader.snipe_token(
            token_address=token.address,
            sol_amount=self.snipe_amount_sol,
            slippage_bps=self.slippage_bps
        )
        
        if tx_signature:
            # Record position
            self.active_positions[token.address] = {
                'token': token,
                'entry_time': datetime.now(),
                'entry_price': token.price_usd,
                'amount_sol': self.snipe_amount_sol,
                'tx_signature': tx_signature
            }
            active_positions.set(len(self.active_positions))
            logger.info(f"Snipe successful! TX: {tx_signature}")
            return True
        else:
            logger.error(f"Snipe failed for {token.symbol}")
            return False
    
    async def run_scan_cycle(self):
        """Run one scan cycle"""
        try:
            # Scan and score tokens
            scored_tokens = await self.scan_and_score_tokens(limit=50)
            
            if not scored_tokens:
                logger.info("No tokens found matching criteria")
                return
            
            # Log top tokens
            logger.info(f"\n=== Top Tokens ===")
            for i, (token, twitter_trend, score) in enumerate(scored_tokens[:10], 1):
                logger.info(f"{i}. {token.symbol} ({token.address[:8]}) - Score: {score:.2f}")
                logger.info(f"   Price: ${token.price_usd:.8f}, Liquidity: ${token.liquidity:,.0f}")
                logger.info(f"   Twitter: {twitter_trend.mentions_count} mentions, "
                          f"Engagement: {twitter_trend.engagement_rate:.2f}")
            
            # Execute snipes on top tokens
            for token, twitter_trend, score in scored_tokens[:5]:  # Top 5
                if len(self.active_positions) >= self.max_positions:
                    break
                
                # Only snipe if score is above threshold
                if score >= self.config.get('min_score_threshold', 50.0):
                    await self.execute_snipe(token)
                    await asyncio.sleep(2)  # Rate limiting
        except Exception as e:
            logger.error(f"Error in scan cycle: {e}")
            import traceback
            logger.debug(traceback.format_exc())
    
    async def run(self, scan_interval_seconds: int = 60):
        """Run the bot continuously"""
        logger.info(f"Starting bot with {scan_interval_seconds}s scan interval")
        
        while True:
            try:
                await self.run_scan_cycle()
                logger.info(f"Waiting {scan_interval_seconds}s until next scan...")
                await asyncio.sleep(scan_interval_seconds)
            except KeyboardInterrupt:
                logger.info("Bot stopped by user")
                break
            except Exception as e:
                logger.error(f"Error in main loop: {e}")
                await asyncio.sleep(scan_interval_seconds)


# Example usage and configuration
def main():
    """Main entry point"""
    # Load configuration from environment or config file
    config = {
        # Solana configuration
        'solana_private_key': os.getenv('SOLANA_PRIVATE_KEY', ''),
        'solana_rpc_url': os.getenv('SOLANA_RPC_URL', 'https://api.mainnet-beta.solana.com'),
        
        # Twitter configuration
        'twitter_bearer_token': os.getenv('TWITTER_BEARER_TOKEN', ''),
        'twitter_api_key': os.getenv('TWITTER_API_KEY', ''),
        'twitter_api_secret': os.getenv('TWITTER_API_SECRET', ''),
        'twitter_access_token': os.getenv('TWITTER_ACCESS_TOKEN', ''),
        'twitter_access_token_secret': os.getenv('TWITTER_ACCESS_TOKEN_SECRET', ''),
        
        # Trading parameters
        'min_liquidity': float(os.getenv('MIN_LIQUIDITY', '50000')),
        'max_market_cap': float(os.getenv('MAX_MARKET_CAP', '10000000')),
        'min_volume_24h': float(os.getenv('MIN_VOLUME_24H', '10000')),
        'max_age_hours': float(os.getenv('MAX_AGE_HOURS', '24')),
        'min_twitter_mentions': int(os.getenv('MIN_TWITTER_MENTIONS', '10')),
        'snipe_amount_sol': float(os.getenv('SNIPE_AMOUNT_SOL', '0.1')),
        'max_positions': int(os.getenv('MAX_POSITIONS', '5')),
        'slippage_bps': int(os.getenv('SLIPPAGE_BPS', '500')),
        'min_score_threshold': float(os.getenv('MIN_SCORE_THRESHOLD', '50.0')),
    }
    
    # Start Prometheus metrics server
    try:
        start_http_server(8000)
        logger.info("Prometheus metrics server started on port 8000")
    except Exception as e:
        logger.warning(f"Could not start metrics server: {e}")
    
    # Initialize and run bot
    bot = MemecoinSniperBot(config)
    
    # Run bot
    scan_interval = int(os.getenv('SCAN_INTERVAL_SECONDS', '60'))
    asyncio.run(bot.run(scan_interval_seconds=scan_interval))


if __name__ == "__main__":
    main()

